# Setup and imports

In [ ]:
# imports
import numpy as np
from tueplots import bundles, figsizes
import wandb
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import pandas as pd


import sys

%load_ext autoreload
%autoreload 2

sys.path.insert(0, '.')

In [ ]:
from analysis import sweep2df, plot_typography, stats2string, RED, BLUE

In [ ]:
USETEX = True

In [ ]:
plt.rcParams.update(bundles.icml2022(usetex=USETEX))
# plt.rcParams.update({
#     'text.latex.preamble': [r'\usepackage{amsfonts}', # mathbb
#                             r'\usepackage{amsmath}'] # boldsymbol
# })

In [ ]:
plot_typography(usetex=USETEX, small=12, medium=16, big=20)


In [ ]:
# Constants
ENTITY = "causal-representation-learning"
PROJECT = "rule_extrapolation"

# W&B API
api = wandb.Api(timeout=200)
runs = api.runs(ENTITY + "/" + PROJECT)

# Data loading

## baN

In [ ]:
SWEEP_ID = "y2193u7e"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
filename = f"ban_{SWEEP_ID}"
ban_df,ban_train_loss,ban_val_loss,ban_val_kl,ban_val_accuracy,ban_finised,ban_ood_finised,ban_sos_finised,ban_r1,ban_r2,ban_grammatical,ban_ood_r1,ban_ood_r1_completion,ban_ood_r2,ban_ood_grammatical,ban_sos_r1,ban_sos_r2,ban_sos_grammatical= sweep2df(sweep.runs, filename, save=True, load=False)

## bbaN

In [ ]:
SWEEP_ID = "hup2v4ru"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
filename = f"bban_{SWEEP_ID}"
bban_df,bban_train_loss,bban_val_loss,bban_val_kl,bban_val_accuracy,bban_finised,bban_ood_finised,bban_sos_finised,bban_r1,bban_r2,bban_grammatical,bban_ood_r1,bban_ood_r1_completion,bban_ood_r2,bban_ood_grammatical,bban_sos_r1,bban_sos_r2,bban_sos_grammatical= sweep2df(sweep.runs, filename, save=True, load=False)

## aNbN

In [ ]:
SWEEP_ID = "nfrfpkqm"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
filename = f"anbn_{SWEEP_ID}"
anbn_df,anbn_train_loss,anbn_val_loss,anbn_val_kl,anbn_val_accuracy,anbn_finised,anbn_ood_finised,anbn_sos_finised,anbn_r1,anbn_r2,anbn_grammatical,anbn_ood_r1,anbn_ood_r1_completion,anbn_ood_r2,anbn_ood_grammatical,anbn_sos_r1,anbn_sos_r2,anbn_sos_grammatical= sweep2df(sweep.runs, filename, save=True, load=False)

## aNbNcN

In [ ]:
SWEEP_ID = "6m1qb70e"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
filename = f"anbncn_{SWEEP_ID}"
anbncn_df,anbncn_train_loss,anbncn_val_loss,anbncn_val_kl,anbncn_val_accuracy,anbncn_finised,anbncn_ood_finised,anbncn_sos_finised,anbncn_r1,anbncn_r2,anbncn_grammatical,anbncn_ood_r1,anbncn_ood_r1_completion,anbncn_ood_r2,anbncn_ood_grammatical,anbncn_sos_r1,anbncn_sos_r2,anbncn_sos_grammatical= sweep2df(sweep.runs, filename, save=True, load=False)

## Matched brackets and parentheses

In [ ]:
SWEEP_ID = "r9wywplg"
sweep = api.sweep(f"{ENTITY}/{PROJECT}/{SWEEP_ID}")
filename = f"brackets_{SWEEP_ID}"
brackets_df,brackets_train_loss,brackets_val_loss,brackets_val_kl,brackets_val_accuracy,brackets_finised,brackets_ood_finised,brackets_sos_finised,brackets_r1,brackets_r2,brackets_grammatical,brackets_ood_r1,brackets_ood_r1_completion,brackets_ood_r2,brackets_ood_grammatical,brackets_sos_r1,brackets_sos_r2,brackets_sos_grammatical= sweep2df(sweep.runs, filename, save=True, load=False)

# Plots

In [ ]:
# group by model

def grouped_rule_stats(df):
    grouped_df = df.groupby("model")

    stats = {}

    stats["val_loss"] = grouped_df.min_val_loss

    stats["rule_1"] = grouped_df.rule_1_accuracy4min_val_loss
    stats["rule_2"] = grouped_df.rule_2_accuracy4min_val_loss
    stats["grammatical"] = grouped_df.grammatical_accuracy4min_val_loss
    stats["finished"] = grouped_df.finished4min_val_loss

    stats["ood_rule_1"] = grouped_df.ood_rule_1_accuracy4min_val_loss
    stats["ood_rule_2"] = grouped_df.ood_rule_2_accuracy4min_val_loss
    stats["ood_rule_2_completion"] = grouped_df.ood_rule_2_completion_accuracy4min_val_loss
    stats["ood_grammatical"] = grouped_df.ood_grammatical_accuracy4min_val_loss
    stats["ood_finished"] = grouped_df.ood_finished4min_val_loss

    stats["sos_rule_1"] = grouped_df.sos_rule_1_accuracy4min_val_loss
    stats["sos_rule_2"] = grouped_df.sos_rule_2_accuracy4min_val_loss
    stats["sos_grammatical"] = grouped_df.sos_grammatical_accuracy4min_val_loss
    stats["sos_finished"] = grouped_df.sos_finished4min_val_loss

    return stats


In [ ]:
ban_stats = grouped_rule_stats(ban_df)
anbn_stats = grouped_rule_stats(anbn_df)
anbncn_stats = grouped_rule_stats(anbncn_df)
brackets_stats = grouped_rule_stats(brackets_df)

## baN

In [ ]:
TICK_PADDING = 2
LABELPAD = 1
cmap = "coolwarm"

stats = ban_stats

fig = plt.figure(figsize=figsizes.icml2022_full(nrows=1, ncols=2)['figure.figsize'])

ax = fig.add_subplot(121)
# ax.grid(True, which="both", ls="-.")
# ax.set_axisbelow(True)

x_pos = list(range(len(stats["rule_1"])))

ax.errorbar(x_pos, 100*stats["rule_1"].mean(), yerr=10*stats["rule_1"].std(), fmt='o', label="R1")
ax.errorbar(x_pos, 100*stats["rule_2"].mean(), yerr=10*stats["rule_2"].std(), fmt='o', label="R2")
ax.errorbar(x_pos, 100*stats["grammatical"].mean(), yerr=10*stats["grammatical"].std(), fmt='o', label="R1+R2")
# ax.errorbar(x_pos, 100*stats["finished"].mean(), yerr=10*stats["finished"].std(), fmt='o', label="F")

ax.set_ylabel("Accuracy (\%)", labelpad=LABELPAD)

# set xtick names
ax.set_xticks(x_pos)
ax.set_xticklabels(stats["rule_1"].groups.keys())

ax2 = fig.add_subplot(122)

ax2.errorbar(x_pos, 100*stats["ood_rule_1"].mean(), yerr=10*stats["ood_rule_1"].std(), fmt='o', label="R1")
ax2.errorbar(x_pos, 100*stats["ood_rule_2"].mean(), yerr=10*stats["ood_rule_2"].std(), fmt='o', label="R2")
ax2.errorbar(x_pos, 100*stats["ood_rule_2_completion"].mean(), yerr=10*stats["ood_rule_2_completion"].std(), fmt='o', label="R2 (compl)")


# set xtick names
ax2.set_xticks(x_pos)
ax2.set_xticklabels(stats["rule_1"].groups.keys())
ax2.set_ylabel("Accuracy (\%)", labelpad=LABELPAD)


plt.legend()
ax.tick_params(axis='both', which='major', pad=TICK_PADDING)


## aNbN

In [ ]:
anbn_stats["rule_2"].mean()

In [ ]:
TICK_PADDING = 2
LABELPAD = 1
cmap = "coolwarm"

stats = anbn_stats

fig = plt.figure(figsize=figsizes.icml2022_full(nrows=1, ncols=2)['figure.figsize'])

ax = fig.add_subplot(121)
# ax.grid(True, which="both", ls="-.")
# ax.set_axisbelow(True)

x_pos = list(range(len(stats["rule_1"])))

ax.errorbar(x_pos, 100*stats["rule_1"].mean(), yerr=10*stats["rule_1"].std(), fmt='o', label="R1")
ax.errorbar(x_pos, 100*stats["rule_2"].mean(), yerr=10*stats["rule_2"].std(), fmt='o', label="R2")
ax.errorbar(x_pos, 100*stats["grammatical"].mean(), yerr=10*stats["grammatical"].std(), fmt='o', label="R1+R2")
# ax.errorbar(x_pos, 100*stats["finished"].mean(), yerr=10*stats["finished"].std(), fmt='o', label="F")

ax.set_ylabel("Accuracy (\%)", labelpad=LABELPAD)

# set xtick names
ax.set_xticks(x_pos)
ax.set_xticklabels(stats["rule_1"].groups.keys())

ax2 = fig.add_subplot(122)

ax2.errorbar(x_pos, 100*stats["ood_rule_1"].mean(), yerr=10*stats["ood_rule_1"].std(), fmt='o', label="R1")
ax2.errorbar(x_pos, 100*stats["ood_rule_2"].mean(), yerr=10*stats["ood_rule_2"].std(), fmt='o', label="R2")
ax2.errorbar(x_pos, 100*stats["ood_rule_2_completion"].mean(), yerr=10*stats["ood_rule_2_completion"].std(), fmt='o', label="R2 (compl)")


# set xtick names
ax2.set_xticks(x_pos)
ax2.set_xticklabels(stats["rule_1"].groups.keys())
ax2.set_ylabel("Accuracy (\%)", labelpad=LABELPAD)


plt.legend()
ax.tick_params(axis='both', which='major', pad=TICK_PADDING)


# plt.savefig("adversarial_rule_extrapolation.svg")

## aNbNcN

In [ ]:
TICK_PADDING = 2
LABELPAD = 1
cmap = "coolwarm"

stats = anbncn_stats

fig = plt.figure(figsize=figsizes.icml2022_full(nrows=1, ncols=2)['figure.figsize'])

ax = fig.add_subplot(121)
# ax.grid(True, which="both", ls="-.")
# ax.set_axisbelow(True)

x_pos = list(range(len(stats["rule_1"])))

ax.errorbar(x_pos, 100*stats["rule_1"].mean(), yerr=10*stats["rule_1"].std(), fmt='o', label="R1")
ax.errorbar(x_pos, 100*stats["rule_2"].mean(), yerr=10*stats["rule_2"].std(), fmt='o', label="R2")
ax.errorbar(x_pos, 100*stats["grammatical"].mean(), yerr=10*stats["grammatical"].std(), fmt='o', label="R1+R2")
# ax.errorbar(x_pos, 100*stats["finished"].mean(), yerr=10*stats["finished"].std(), fmt='o', label="F")

ax.set_ylabel("Accuracy (\%)", labelpad=LABELPAD)

# set xtick names
ax.set_xticks(x_pos)
ax.set_xticklabels(stats["rule_1"].groups.keys())

ax2 = fig.add_subplot(122)

ax2.errorbar(x_pos, 100*stats["ood_rule_1"].mean(), yerr=10*stats["ood_rule_1"].std(), fmt='o', label="R1")
ax2.errorbar(x_pos, 100*stats["ood_rule_2"].mean(), yerr=10*stats["ood_rule_2"].std(), fmt='o', label="R2")
ax2.errorbar(x_pos, 100*stats["ood_rule_2_completion"].mean(), yerr=10*stats["ood_rule_2_completion"].std(), fmt='o', label="R2 (compl)")


# set xtick names
ax2.set_xticks(x_pos)
ax2.set_xticklabels(stats["rule_1"].groups.keys())
ax2.set_ylabel("Accuracy (\%)", labelpad=LABELPAD)


plt.legend()
ax.tick_params(axis='both', which='major', pad=TICK_PADDING)


## Matched brackets and parentheses

In [ ]:
TICK_PADDING = 2
LABELPAD = 1
cmap = "coolwarm"

stats = brackets_stats

fig = plt.figure(figsize=figsizes.icml2022_full(nrows=1, ncols=2)['figure.figsize'])

ax = fig.add_subplot(121)
# ax.grid(True, which="both", ls="-.")
# ax.set_axisbelow(True)

x_pos = list(range(len(stats["rule_1"])))

ax.errorbar(x_pos, 100*stats["rule_1"].mean(), yerr=10*stats["rule_1"].std(), fmt='o', label="R1")
ax.errorbar(x_pos, 100*stats["rule_2"].mean(), yerr=10*stats["rule_2"].std(), fmt='o', label="R2")
ax.errorbar(x_pos, 100*stats["grammatical"].mean(), yerr=10*stats["grammatical"].std(), fmt='o', label="R1+R2")
# ax.errorbar(x_pos, 100*stats["finished"].mean(), yerr=10*stats["finished"].std(), fmt='o', label="F")

ax.set_ylabel("Accuracy (\%)", labelpad=LABELPAD)

# set xtick names
ax.set_xticks(x_pos)
ax.set_xticklabels(stats["rule_1"].groups.keys())

ax2 = fig.add_subplot(122)

ax2.errorbar(x_pos, 100*stats["ood_rule_1"].mean(), yerr=10*stats["ood_rule_1"].std(), fmt='o', label="R1")
ax2.errorbar(x_pos, 100*stats["ood_rule_2"].mean(), yerr=10*stats["ood_rule_2"].std(), fmt='o', label="R2")
ax2.errorbar(x_pos, 100*stats["ood_rule_2_completion"].mean(), yerr=10*stats["ood_rule_2_completion"].std(), fmt='o', label="R2 (compl)")


# set xtick names
ax2.set_xticks(x_pos)
ax2.set_xticklabels(stats["rule_1"].groups.keys())
ax2.set_ylabel("Accuracy (\%)", labelpad=LABELPAD)


plt.legend()
ax.tick_params(axis='both', which='major', pad=TICK_PADDING)


In [ ]:
brackets_stats["val_loss"].mean()


In [ ]:
brackets_df[brackets_df.model =="linear"].min_val_loss

In [ ]:
brackets_stats["val_loss"].groups.keys()

In [ ]:
TICK_PADDING = 2
LABELPAD = 1
cmap = "coolwarm"
df = brackets_df
stats = brackets_stats

fig = plt.figure(figsize=figsizes.icml2022_full(nrows=1, ncols=2)['figure.figsize'])

ax = fig.add_subplot(121)
ax.grid(True, which="both", ls="-.")
ax.set_axisbelow(True)

for model in stats["val_loss"].groups.keys():
    df[df.model==model].min_val_loss
    im = ax.scatter(df[df.model==model].min_val_loss, 100*df[df.model==model].ood_rule_1_accuracy4min_val_loss, cmap=cmap, label=model)


ax.set_ylabel("R1 \%", labelpad=LABELPAD)
ax.set_xlabel("Minimum test loss", labelpad=LABELPAD)
# plt.legend()
ax.tick_params(axis='both', which='major', pad=TICK_PADDING)

ax = fig.add_subplot(122)
ax.grid(True, which="both", ls="-.")
ax.set_axisbelow(True)

for model in stats["val_loss"].groups.keys():
    df[df.model==model].min_val_loss
    im = ax.scatter(df[df.model==model].min_val_loss, 100*df[df.model==model].ood_rule_2_completion_accuracy4min_val_loss, cmap=cmap, label=model)


ax.set_ylabel("R2 completion (\%)", labelpad=LABELPAD)
ax.set_xlabel("Minimum test loss", labelpad=LABELPAD)
plt.legend(loc='lower center')
ax.tick_params(axis='both', which='major', pad=TICK_PADDING)


# plt.savefig("adversarial_rule_extrapolation.svg")